In [4]:
import pandas as pd
import mysql.connector

# 1. MySQL 연결
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="분석실습"
)
cursor = conn.cursor()



In [5]:
# 2. 파일에서 데이터프레임으로 읽기 (탭 구분자라고 가정)
df_customer = pd.read_csv("./mig/customer.txt", sep="\t", dtype=str,  encoding='cp949')
df_customer
#(5369, 8)


,mem_no,last_name,first_name,gd,birth_dt,entr_dt,grade,sign_up_ch
0,18466,정,하성,M,1988-03-11,2011-01-01,VVIP,01
1,18798,정,호정,F,1990-02-28,2013-03-09,VVIP,01
2,18434,황,성연,F,1971-04-04,2011-09-12,VIP,03
3,18772,홍,경빈,F,1971-07-06,2012-08-18,VIP,03
4,18326,조,우준,M,1972-05-08,2011-05-26,VIP,01
...,...,...,...,...,...,...,...,...
5364,16332,정,윤제,M,1982-06-03,2012-02-23,BRONZE,01
5365,17987,임,민건,M,1992-04-16,2013-03-18,BRONZE,04
5366,17894,임,정윤,F,1992-04-22,2012-06-03,BRONZE,06
5367,14578,김,로건,M,1992-04-23,2013-10-20,BRONZE,02


In [6]:
df_sales = pd.read_csv("mig/sales.txt", sep="\t", dtype=str,  encoding='cp949')
df_sales.shape
#(541909, 8)

(541909, 8)

In [7]:
# 3. CUSTOMER INSERT
for row in df_customer.itertuples(index=False, name=None):
    cursor.execute("""
        INSERT INTO CUSTOMER (mem_no, last_name, first_name, gd, birth_dt, entr_dt, grade, sign_up_ch)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, row)
    
conn.commit()  # 변경사항 저장

cursor.execute("SELECT COUNT(*) FROM CUSTOMER")
print(f"총 {cursor.fetchone()[0]}건 삽입 완료")

총 5369건 삽입 완료


In [8]:
# 4. SALES 데이터전처리
df_sales = df_sales.dropna(subset=["CustomerID"])
df_sales["CustomerID"] = df_sales["CustomerID"].astype(int)

# 날짜 및 숫자 변환
df_sales["Quantity"] = df_sales["Quantity"].astype(int)
df_sales["UnitPrice"] = df_sales["UnitPrice"].astype(float)
df_sales["CustomerID"] = df_sales["CustomerID"].astype(int)

# DATETIME 변환
df_sales["InvoiceDate"] = pd.to_datetime(df_sales["InvoiceDate"], errors='coerce')
df_sales["InvoiceDate"] = df_sales["InvoiceDate"].dt.strftime('%Y-%m-%d %H:%M:%S')
df_sales.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [9]:
# 5. SALES INSERT
df_sales = df_sales.astype(object)
rows = [tuple(x) for x in df_sales.to_records(index=False)]
cursor.executemany("""
    INSERT INTO SALES (InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
""", rows)

conn.commit()

cursor.execute("SELECT COUNT(*) FROM SALES")
print(f"총 {cursor.fetchone()[0]}건 삽입 완료")

총 406829건 삽입 완료


In [10]:
# 6. 종료
cursor.close()
conn.close()
print("데이터 삽입 완료")


데이터 삽입 완료
